In [33]:
import praw
import pandas as pd
# from praw.models import MoreComments #This import is needed is used only if we want to limit the comments we scrape

# #URL to access the app needed to scrape the data off the Berkeley subreddit
# # https://www.reddit.com/prefs/apps

#This is the tutorial I used to set up the web scraping using PRAW
#https://www.geeksforgeeks.org/python/scraping-reddit-using-python/

# #PLAN
# #Scrape the data off the Berkeley subreddit about consulting clubs
# #Organize that data using the pandas library
# #Create chatbot that utilizes natural language processing that will give users feedback
# #about each consulting club here at Berkeley. 

reddit_read_only = praw.Reddit(client_id = "QlBfNfxQ3e_MGP9RkaOQig",
                               client_secret = "SpLjOwYdQPU4z1wqcXBjVl_7DnUIZg",
                               user_agent = "Berkeley_Consulting")

subreddit = reddit_read_only.subreddit("berkeley")

# df1 = pd.DataFrame({
#     'post_id': ['p1', 'p2', 'p3'],
#     'title': ['Title A', 'Title B', 'Title C'],
#     'author': ['user1', 'user2', 'user3']
# })

# df2 = pd.DataFrame({
#     'comment_id': ['c1', 'c2', 'c3', 'c4', 'c5'],
#     'post_id': ['p1', 'p1', 'p2', 'p2', 'p4'],  # Note: p4 doesn't exist in df1
#     'comment': ['Great post!', 'Thanks!', 'Interesting', 'I agree', 'Nice!']
# })
# outer_merged = df1.merge(df2, on='post_id', how='outer')
# print(outer_merged)

In [34]:
consulting_posts = subreddit.search('consulting')

posts_dict = {'Title': [], 'Post Text': [], 'ID': [], 'Score': [], 'Total Comments': [], 'Post URL': []}
comments_dict = {'Comment': [], 'Score': []}

In [35]:
for post in consulting_posts:
    if len(post.selftext) > 20:
        #The title of the post
        posts_dict['Title'].append(post.title)
        #The text inside of the post
        posts_dict['Post Text'].append(post.selftext)
        #Unique ID of each post
        posts_dict['ID'].append(post.id)
        #The scoure of a post
        posts_dict['Score'].append(post.score)
        #Total number of comments inside the post
        posts_dict['Total Comments'].append(post.num_comments)
        #URL of each post
        posts_dict['Post URL'].append(post.url)
    else:
        continue

consulting_club_posts = pd.DataFrame(posts_dict)
# print(consulting_club_posts)
consulting_club_posts

,Title,Post Text,ID,Score,Total Comments,Post URL
0,Asian monoculture in consulting clubs,Over the past month on this subreddit there ha...,16o5u8z,268,112,https://www.reddit.com/r/berkeley/comments/16o...
1,What are consulting clubs actually looking for?,I am an incoming freshman at Haas next year an...,1kvi9jt,44,55,https://www.reddit.com/r/berkeley/comments/1kv...
2,Tech PM blocks all “.berkeley.edu” e-mails bc ...,Consulting clubs making us look bad smh,19d8ceg,558,43,https://i.redd.it/em6d28vzh2ec1.jpeg
3,Rejected from consulting club after having bee...,titles kinda self explanatory lol. didn’t even...,1na99k8,77,22,https://www.reddit.com/r/berkeley/comments/1na...
4,Why are some of these Business/Consulting Club...,My friend and I were enjoying dinner in the so...,16jsy19,366,55,https://www.reddit.com/r/berkeley/comments/16j...
...,...,...,...,...,...,...
80,got into a consulting club that has nothing to...,do i send it? do i not? help.,sn6pop,24,5,https://www.reddit.com/r/berkeley/comments/sn6...
81,another bear rejected by consulting clubs,I know it's just a club but man I wanna cry i ...,piudil,33,7,https://www.reddit.com/r/berkeley/comments/piu...
82,NEW! - Stress Management Consultations,Stress Management Consultations at the UHS Car...,zdf1h4,6,0,https://www.reddit.com/r/berkeley/comments/zdf...
83,Made it to a Consulting Club’s Final Round Int...,I’m not happy about getting rejected but I did...,sjnb6y,2,6,https://www.reddit.com/r/berkeley/comments/sjn...


In [36]:
#Need to figure out a way to get the URL of each and every post
for i in posts_dict['Post URL']:
    if 'comments' in i:
        submission = reddit_read_only.submission(url = i)
    else:
        continue
    for comment in submission.comments:
        if len(comment.body) < 20:
            continue
        comments_dict['Comment'].append(comment.body)
        comments_dict['Score'].append(comment.score)

# print(post_comments[0]) #This only prints one comment right now for one URL. Need to get as many comments as possible for one given URL.

comments_df = pd.DataFrame(comments_dict)
comments_df

#Generate the CSV file that contains all the posts about Berkeley clubs
# comments_df.to_csv('my_dataframe.csv', index=False) # index=False prevents writing the DataFrame index as a column

#They are not marked in any way so they are just random pieces of information
#Need to find a way to associate either the post title or the post URL with the post comments

,Comment,Score
0,Diversatech at Berkeley is the worst one,136
1,Bro there is asian monoculture at the school lol,179
2,"Interesting take and valid points, but i think...",148
3,for the pbl one they have basically a DEI stat...,65
4,sucks but it’s how it be sometimes - you can m...,64
...,...,...
514,"Hey there, just went through the other side of...",2
515,Hi! I went through all the interview rounds bu...,1
516,Start your own if you really want to be one of...,1
517,Tailor it to the club as much as possible. We ...,1


In [37]:
new_df = pd.concat([consulting_club_posts['Post Text'], comments_df['Comment']], axis=1)
new_df

,Post Text,Comment
0,Over the past month on this subreddit there ha...,Diversatech at Berkeley is the worst one
1,I am an incoming freshman at Haas next year an...,Bro there is asian monoculture at the school lol
2,Consulting clubs making us look bad smh,"Interesting take and valid points, but i think..."
3,titles kinda self explanatory lol. didn’t even...,for the pbl one they have basically a DEI stat...
4,My friend and I were enjoying dinner in the so...,sucks but it’s how it be sometimes - you can m...
...,...,...
514,NaN,"Hey there, just went through the other side of..."
515,NaN,Hi! I went through all the interview rounds bu...
516,NaN,Start your own if you really want to be one of...
517,NaN,Tailor it to the club as much as possible. We ...


In [38]:
# #Loading the stored dataframes from the previous notebook using the pickle module
# consulting_club_posts.to_pickle('consulting_club_posts.pkl')
# comments_df.to_pickle('comments_df.pkl')

In [39]:
new_df.to_csv('posts_and_comments.csv', index=False)

In [40]:
# This is test code if nothing is working
# #Display the name of the subreddit
# print("Display Name:", subreddit.display_name)

# #Display the title of the subreddit
# print("Title:", subreddit.title)

# #Display the description of the subreddit
# print("Description:", subreddit.description)

In [41]:
# subreddit._fetch()

# print(sorted(vars(subreddit).keys()))

# print("Display Name:", subreddit.display_name)

# try:
#     print("Title:", subreddit.unsubscribe)
# except Exception as e:
#     print("Error:", e)

# print("Title:", subreddit.title)

# print("Description:", subreddit.description)